# Patch Metrics

This notebook processes the raw patch areas exported from Earth Engine into a CSV for use in the time series processing script.

In [ ]:
import pandas as pd

from pfh.scripts.config import OWNER_CLASSES, OWNER_GROUPS

In [ ]:
# Special class that was added for patch areas across ownerships, since they can't be
# aggregated
OWNER_CLASSES["all"] = 99
OWNER_CODES = {v: k for k, v in OWNER_CLASSES.items()}

In [ ]:
patch_metrics = pd.read_csv("../data/results/patch_metrics.csv")
patch_metrics["owner"] = patch_metrics.label.apply(lambda x: OWNER_CODES[x])
patch_metrics["owner_group"] = patch_metrics.owner.apply(lambda x: OWNER_GROUPS[x])

patch_metrics = patch_metrics.drop(
    columns=["system:index", ".geo", "label", "event_id", "owner"]
)

patch_trends = patch_metrics.groupby(["year", "owner_group"]).agg("mean").reset_index()

In [ ]:
# Format the trends so they match the previous output
export_patch_trends = patch_trends[patch_trends.owner_group.ne("Other")].rename(
    columns={"owner_group": "owner"}
)
export_patch_trends["owner"] = export_patch_trends.owner.apply(
    lambda x: "Total" if x == "All owners" else x
)

In [ ]:
export_patch_trends.to_csv("../data/results/annual_mean_patch_area.csv", index=False)